# Final Project - IR

Name: Wenjie Jin <br>
NIA: 219516

https://developer.twitter.com/en/apps

In [1]:
#Import librery
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *

#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wenjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## access token informations 
access_token1 = "1240651902712516613-BF3CepSxPUoMr3lDkptNt08f2GwaGZ"
access_token_secret1 = "79ga9EksEgtgpHGJhXy7A4ujb31pw1AjwP8lOlaYtmDdk"

consumer_key1 = "ieusj8Wof83kmsVe9PDLJQqn3"
consumer_secret1 = "EpZYsMfEqvHSwFQyAfz2wCdVujPTtr7JQtvkBt1g9sMg5t0m3B"

In [3]:
#Generate API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

##### Store File in Json. 

In [7]:
%%time

OUTPUT_FILENAME = "../other-outputs/output.json"
stop_condition = 10000

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["coronavirus", "covid", "#COVID", "#coronavirus", "#COVID19", "recovery cases", "suspected cases", "death cases", "vaccines", "disinfection"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

ProtocolError: ('Connection broken: IncompleteRead(7623 bytes read)', IncompleteRead(7623 bytes read))

##### Load data from Json

In [8]:
#load data from output.json scrapped from twitter
with open("../other-outputs/output.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

df_tweets = pd.DataFrame.from_records(data)
df_tweets

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,extended_tweet,display_text_range,extended_entities
0,Sat Dec 05 10:11:06 +0000 2020,1335164788498980864,1335164788498980864,RT @JoeDanMedia: In 2009... Santelli lit the T...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Dec 05 10:11:06 +0000 2020,1335164788767277061,1335164788767277061,RT @CNNPolitics: President-elect Joe Biden on ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,None,NaN,None,...,en,1607163066461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Dec 05 10:11:06 +0000 2020,1335164789060984832,1335164789060984832,Arbroath #arbroath #covid,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066531,1.335162e+18,1335162235216420864,{'created_at': 'Sat Dec 05 10:00:57 +0000 2020...,"{'url': 'https://t.co/HrzZRGUtCS', 'expanded':...",NaN,NaN,NaN,NaN
3,Sat Dec 05 10:11:06 +0000 2020,1335164790180864001,1335164790180864001,RT @johnnymorls: I think that wraps things up ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066798,1.335132e+18,1335132252800503808,{'created_at': 'Sat Dec 05 08:01:49 +0000 2020...,"{'url': 'https://t.co/738ZoPEBv7', 'expanded':...",NaN,NaN,NaN,NaN
4,Sat Dec 05 10:11:06 +0000 2020,1335164790365368320,1335164790365368320,RT @CAOYI170610: According to a recent US CDC ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1607163066842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4607,Thu Dec 17 15:32:40 +0000 2020,1339594369616150529,1339594369616150529,In Memoriam: Healthcare Workers Who Have Died ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1608219160833,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
4608,Thu Dec 17 15:32:40 +0000 2020,1339594369632849924,1339594369632849924,@friendlyfire78 @sfdc_paul @AllergyKidsDoc Tha...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.339592e+18,1339591721852698625,2.659670e+09,2659670162,...,en,1608219160837,NaN,NaN,NaN,NaN,NaN,{'full_text': '@friendlyfire78 @sfdc_paul @All...,"[43, 140]",NaN
4609,Thu Dec 17 15:32:40 +0000 2020,1339594369326665736,1339594369326665736,RT @HuffmanForNC: #ComeOnTed your family has $...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1608219160764,1.339575e+18,1339575478873677825,{'created_at': 'Thu Dec 17 14:17:36 +0000 2020...,"{'url': 'https://t.co/TApeFjPrqU', 'expanded':...",NaN,NaN,NaN,NaN
4610,Thu Dec 17 15:32:40 +0000 2020,1339594369603551235,1339594369603551235,InstituteForPR: With vaccine hesitancy on the ...,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",True,NaN,None,NaN,None,...,en,1608219160830,NaN,NaN,NaN,NaN,False,{'full_text': 'InstituteForPR: With vaccine he...,NaN,NaN


###### Show tweets collection information

In [9]:
def print_retweets_uniqueTweets_uniqueUsers(df_tweets):
    retweets_ = df_tweets["retweeted_status"].apply(lambda x: 0 if str(x) == "nan" else 1)
    tot_retweets = sum(retweets_)
    unique_tweets = len(retweets_) - tot_retweets
    
    users_ = df_tweets.user.apply(lambda x: x["id"])
    
    tot_users = len(set(users_))
    
    print("Total Retweets:", tot_retweets)

    print("Unique Tweets:", unique_tweets)
    
    print("Unique Users:", tot_users)

In [10]:
print_retweets_uniqueTweets_uniqueUsers(df_tweets)

Total Retweets: 3040
Unique Tweets: 1572
Unique Users: 4128


In [11]:
# Get unique tweets
tweets=df_tweets[df_tweets["retweeted_status"].isna()]
tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,extended_tweet,display_text_range,extended_entities
0,Sat Dec 05 10:11:06 +0000 2020,1335164788498980864,1335164788498980864,RT @JoeDanMedia: In 2009... Santelli lit the T...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Dec 05 10:11:06 +0000 2020,1335164788767277061,1335164788767277061,RT @CNNPolitics: President-elect Joe Biden on ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,None,NaN,None,...,en,1607163066461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Dec 05 10:11:06 +0000 2020,1335164789060984832,1335164789060984832,Arbroath #arbroath #covid,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066531,1.335162e+18,1335162235216420864,{'created_at': 'Sat Dec 05 10:00:57 +0000 2020...,"{'url': 'https://t.co/HrzZRGUtCS', 'expanded':...",NaN,NaN,NaN,NaN
3,Sat Dec 05 10:11:06 +0000 2020,1335164790180864001,1335164790180864001,RT @johnnymorls: I think that wraps things up ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066798,1.335132e+18,1335132252800503808,{'created_at': 'Sat Dec 05 08:01:49 +0000 2020...,"{'url': 'https://t.co/738ZoPEBv7', 'expanded':...",NaN,NaN,NaN,NaN
4,Sat Dec 05 10:11:06 +0000 2020,1335164790365368320,1335164790365368320,RT @CAOYI170610: According to a recent US CDC ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1607163066842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
